In [ ]:
###- TESIS Constanza Gaset
# Mayo 2022
# UNIVERSIDAD TORCUATO DI TELLA
# REDES NEURONALES RECURRENTES APLICADAS A SERIES DE TIEMPO: PREDICCIÓN DE PASAJEROS DE RUTAS REGIONALES EN ARGENTINA

In [ ]:
# Comenzamos con las configuraciones generales y las librerías a utilizar
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from pandas.plotting import register_matplotlib_converters %matplotlib inline
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.preprocessing.sequence import TimeseriesGenerator import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [ ]:
# Seteamos del espacio de trabajo, en este caso, un espacio local
os.getcwd()
os.chdir('C:/Users/Desktop/)

In [ ]:
### RED LSTM MULTIVARIADA
# Este proceso se realiza para las 11 rutas bajo análisis.
# Se replica para el año 2020 (como horizonte de proyección), utilizando como entrenamiento 2001 a 2018 y prueba 2019 a 2020.


In [ ]:
# 1) Preprocesamiento de los datos
# Cargamos el dataset, previamente transformado, a nivel ruta, para las 11 rutas en análisis. Se agregaron también los datos de evolución del tipo de cambio y EMAE
df_BUEXXX_lstm_multiv=pd.read_excel('Database.xlsx', heet_name='Dataset_Multiv_BUEXXX', indexcol=0)


In [ ]:
# Para la red multivariada filtramos los datos, quedándonos con las observaciones desde 2004 a 2021
df_BUEXXX_lstm_multiv=df_BUEXXX_lstm_multiv.iloc[:216,1:]

In [ ]:
# Separamos los datasets de entrenamiento y prueba: 2001-2019 entrenamiento y 2020-2021 prueba train_size= 192
train_BUEXXX_lstm_multiv,test_BUEXXX_lstm_multiv=df_BUEXXX_lstm_mult iv.iloc[0:train_size], df_BUEXXX_lstm_multiv.iloc[train_size:len(df_BUEXXX)]

In [ ]:
 # Revisamos que los datos hayan quedado correctamente separados
print(len(train_BUEXXX_lstm_multiv), len(test_BUEXXX_lstm_multiv)) print(train_BUEXXX_lstm_multiv) print(test_BUEXXX_lstm_multiv)

In [ ]:
# Escalamos los datos entre 0 y 1 con MinMaxScaler
f_columns = ['Vuelos_BUEXXX', 'TC', 'EMAE']
f_col_scaler = MinMaxScaler(feature_range=(0, 1))
pax_scaler = MinMaxScaler(feature_range=(0, 1))

train_BUEXXX_lstm_multiv.loc[:,f_columns]=f_col_scaler.fit_transform(train_BUEXXX_lstm_multiv[f_columns])
train_BUEXXX_lstm_multiv['Pax_BUEXXX']=pax_scaler.fit_transform(train_BUEXXX_lstm_multiv[['Pax_BUEXXX']])

test_BUEXXX_lstm_multiv.loc[:,f_columns]=f_col_scaler.fit_transform(test_BUEXXX_lstm_multiv[f_columns]) test_BUEXXX_lstm_multiv['Pax_BUEXXX']=pax_scaler.fit_transform(test_ BUEXXX_lstm_multiv[['Pax_BUEXXX']])

In [ ]:
# Revisamos que los datos hayan quedado correctamente convertidos
print(train_BUEXXX_lstm_multiv)
print(test_BUEXXX_lstm_multiv)

In [ ]:
# Preparamos la información para la red neuronal con la función create_dataset. A partir de esta función creamos secuencias de datos, definidas por los time steps definidos
def create_dataset(X, y, time_steps=1):
Xs, ys = [], []
for i in range(len(X) - time_steps):
    v = X.iloc[i:(i + time_steps)].values
    Xs.append(v)
    ys.append(y.iloc[i + time_steps])
return np.array(Xs), np.array(ys)

In [ ]:
# Hacemos el reshape aplicando la función, llevamos los datos a un formato [muestras, pasos del tiempo, características]
time_steps = 12
X_train_BUEXXX_lstm_multiv,y_train_BUEXXX_lstm_multiv=create_dataset (train_BUEXXX_lstm_multiv, train_BUEXXX_lstm_multiv.BUEXXX,time_steps)
X_test_BUEXXX_lstm_multiv,y_test_BUEXXX_lstm_multiv=create_dataset(t est_BUEXXX_lstm_multiv, test_BUEXXX_lstm_multiv.BUEXXX, time_steps)

In [ ]:
# Corroboramos que la información es correcta
print(X_train_BUEXXX_lstm_multiv.shape, y_train_BUEXXX_lstm_multiv.shape)
print(X_test_BUEXXX_lstm_multiv.shape, y_test_BUEXXX_lstm_multiv.shape)

In [ ]:
# Observamos las secuencias creadas para entrenamiento y testeo
X_train_BUEXXX_lstm_multiv[0]
y_train_BUEXXX_lstm_multiv[0]
X_train_BUEXXX_lstm_multiv[1] y_train_BUEXXX_lstm_multiv[1]

In [ ]:
# 2) Modelado
# Definimos el modelo a utilizar: red neuronal LSTM secuencial con 1 capa oculta y 100 neuronas, función de perdida entropía cruzada y función de optimización Adam con learning rate 0.01
model = keras.Sequential()
model.add(keras.layers.LSTM(units=100,
input_shape=(X_train_BUEXXX_lstm_multiv.shape[1], X_train_BUEXXX_lstm_multiv.shape[2])))
model.add(keras.layers.Dense(units=1))
model.compile(loss='mean_squared_error', optimizer=keras.optimizers.Adam(0.01))

model.summary()

In [ ]:
# 3) Entrenamiento del modelo
# Para el entrenamiento se definen 100 épocas y tamaño de lote de 42

history_BUEXXX_lstm_multiv = model.fit(
      X_train_BUEXXX_lstm_multiv, y_train_BUEXXX_lstm_multiv,
      epochs=100,
      batch_size=42,
      verbose=1,
      shuffle=False
)

In [ ]:
# Graficamos la perdida en entrenamiento y testeo
plt.plot(history_BUEXXX_lstm_multiv.history['loss'], label='train')
plt.plot(history_BUEXXX_lstm_multiv.history['val_loss'], label='test')
plt.legend()
plt.show()

In [ ]:
# 4) Evaluación del modelo
# Hacemos las proyecciones con el modelo entrenado
y_pred_BUEXXX_lstm_multiv = model.predict(X_test_BUEXXX_lstm_multiv)